In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install python-holidayapi

In [ ]:
import holidayapi
key = 'b444b457-4922-47bf-882b-04357c5cc240'
hapi = holidayapi.v1(key)
holidays = hapi.holidays({
  'country': 'RU',
  'year': '2019',
})

In [ ]:
holidays_df = pd.DataFrame.from_dict(holidays['holidays'])

In [ ]:
get_month = lambda x: int(x[-5:-3])
get_day = lambda x: int(x[-2:])
holidays_df['month'] = holidays_df['date'].apply(get_month)
holidays_df['day'] = holidays_df['date'].apply(get_day)

In [ ]:
holidays_month = holidays_df[['month', 'public','name']].groupby(['month', 'public']).agg('count')

In [ ]:
holidays_month.head(5)

In [ ]:
holidays = pd.pivot_table(holidays_month, values=['name'], index=['month'], columns='public').fillna(0)
holidays.columns = holidays.columns.get_level_values(0)
holidays.columns = ['holidays', 'public_holidays']
holidays['black_friday'] = [0 if(i!=11) else 1 for i in range(1, 13)]
holidays['new_year'] = [0 if(i!=12) else 1 for i in range(1, 13)]
holidays

In [ ]:
holidays.index

In [ ]:
items= pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')

In [ ]:
items[['item_id', 'item_category_id']]

In [ ]:
sales_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
pop_df = pd.DataFrame()
pop_df = sales_train[['shop_id', 'item_id', 'item_price']].merge(items[['item_id', 'item_category_id']], left_on=['item_id'], right_on=['item_id'])

In [ ]:
pop_df

In [ ]:
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
test = test.merge(pop_df, left_on = ['shop_id', 'item_id'], right_on = ['shop_id', 'item_id'])
test['month'] = 11

In [ ]:
test = test.merge(holidays, left_on=['month'], right_on=['month'])

In [ ]:
test['year'] = 2015
test.head()

In [ ]:
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
train['year'] = train['date'].apply(lambda x: int(x[-4:]))
train['month'] = train['date'].apply(lambda x: int(x[3:5]))
train.head()

In [ ]:
sum(train['item_cnt_day'])/(len(train['item_cnt_day']))

In [ ]:
print('min. ' + str(min(train['item_cnt_day'])))
print('max. ' + str(max(train['item_cnt_day']))) 

In [ ]:
zero_the_negatives = lambda x: 0 if(x<0) else x

In [ ]:
train['item_cnt_day'] = train['item_cnt_day'].apply(zero_the_negatives)

In [ ]:
train

In [ ]:
train = train[['shop_id', 'item_id', 'item_price', 'year', 'month', 'item_cnt_day']].groupby(['shop_id', 'item_id', 'item_price', 'year', 'month']).agg('sum')

In [ ]:
train.groupby(['shop_id', 'item_id', 'year', 'month']).agg('mean')

In [ ]:
train = train.reset_index()

In [ ]:
train = train.merge(holidays, left_on=['month'], right_on=['month'])

In [ ]:
import numpy as np
train['item_price_log'] = train['item_price'].apply(np.log)
test['item_price_log'] = test['item_price'].apply(np.log)

In [ ]:
test.head(5)

In [ ]:
test = test.groupby(by=['ID', 'shop_id', 'item_id', 'item_category_id', 'year', 'month', 'holidays', 'public_holidays', 'black_friday', 'new_year']).agg('mean')
test.reset_index()

In [ ]:
len(train)

In [ ]:
train = train.sort_values(by=['year', 'month'])

X_test = train[-10000:].copy()

#remove outliers
X_test = X_test.sort_values(by='item_cnt_day')
X_test = X_test[:-50]

y_test = X_test['item_cnt_day']
X_test = X_test.drop(columns=['item_cnt_day'])

X_val = train[-15000:-10000].copy()
y_val = X_val['item_cnt_day']
X_val = X_val.drop(columns=['item_cnt_day'])


X_train = train[:-15000].copy()
y_train = X_train['item_cnt_day']
X_train = X_train.drop(columns=['item_cnt_day'])

In [ ]:
assert len(X_test) == len(y_test)
assert len(X_train) == len(y_train)
assert len(X_val) == len(y_val)

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
cols_model = ['shop_id', 'item_id', 'year', 'month', 'holidays', 'public_holidays', 'black_friday', 'new_year', 'item_price_log']

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05) # Your code here
my_model.fit(X_train[cols_model], y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_val[cols_model], y_val)], 
             verbose=False)

prediction = my_model.predict(X_test[cols_model])

In [ ]:
mse_1 = mean_squared_error(prediction, y_test)
print(mse_1**0.5)

item_price: 4.3221681585266385

item_price_log: 4.322168179188874

In [ ]:
# Generate calender table
import pandas as pd

def create_date_table(start='2013-01-01', end='2015-11-30'):
    # red days:
    extra_red_days_2013 = [
        '2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07', 
        '2013-01-08', '2013-02-23', '2013-03-08', '2013-05-01', '2013-05-02', 
        '2013-05-03', '2013-05-09', '2013-05-10', '2013-06-12', '2013-11-04']
    extra_red_days_2014 = [
        '2014-01-01', '2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07', 
        '2014-01-08', '2014-02-22', '2014-03-10', '2014-05-01', '2014-05-02', 
        '2014-05-03', '2014-05-09', '2014-05-10', '2014-06-12', '2014-06-13', 
        '2014-06-14', '2014-11-01', '2014-11-03', '2014-11-04']
    extra_red_days_2015 = [
        '2015-01-01', '2015-01-02', '2015-01-03', '2015-01-05', '2015-01-06', 
        '2015-01-07', '2015-01-08', '2015-01-09', '2015-02-23', '2015-03-09',
        '2015-05-01', '2015-05-04', '2015-05-11', '2015-06-12', '2015-11-04']
    extra_red_days = extra_red_days_2013 + extra_red_days_2014 + extra_red_days_2015
    # black fridays
    black_fridays = ['2013-11-29', '2014-11-28', '2015-11-27']
    # create calendet dataframe
    df = pd.DataFrame({"date": pd.date_range(start, end)})
    df["day_name"] = df.date.dt.day_name()
    df["day_of_week"] = df.date.dt.weekday
    df["day_of_month"] = df.date.dt.day
    df["week"] = df.date.dt.weekofyear
    df["month_name"] = df.date.dt.month_name()
    df["month_of_year"] = df.date.dt.month
    df["month_year_name"] = df.date.dt.month_name() + ' ' + df.date.dt.year.astype(str)
    df["month_year_name_sort"] = df.date.dt.year*100 + df.date.dt.month
    df["quarter"] = df.date.dt.quarter
    df["quarter_year_name_sort"] = df.date.dt.year*10 + df.date.dt.quarter
    df["year"] = df.date.dt.year
    df["year_half"] = (df.quarter + 1) // 2
    df["red_day"] = 0
    df.loc[df['day_of_week']==6, "red_day"] = 1
    for extra_red_day in extra_red_days:
        df.loc[df['date'] == extra_red_day, "red_day"] = 1
    df["black_friday"] = 0
    for black_friday in black_fridays:
        df.loc[df['date'] == black_friday, "black_friday"] = 1
        
    return(df)

In [ ]:
calendar = create_date_table()
red_days = calendar[['month_of_year','year','red_day', 'black_friday']].groupby(by=['month_of_year','year']).agg('sum')
red_days=red_days.reset_index()
red_days['month'] = red_days['month_of_year']
red_days = red_days.drop(columns=['month_of_year'])
red_days.head(2)

In [ ]:
train_1 = train.merge(red_days, on=['year', 'month'])

In [ ]:
test_1 =  test.merge(red_days, on=['year', 'month'])

In [ ]:
len(test)

In [ ]:
train_1.columns

In [ ]:
round_2_cols = ['shop_id', 'item_id', 'item_price', 'year', 'month', 'black_friday_x', 'new_year', 'item_price_log', 'red_day']

In [ ]:
train_1 = train_1.sort_values(by=['year', 'month'])

X_test1 = train_1[-10000:].copy()

y_test1 = X_test1['item_cnt_day']
y_test1_log = y_test.apply(lambda x: np.log(x+2))

X_test1 = X_test1.drop(columns=['item_cnt_day'])

X_val1 = train_1[-15000:-10000].copy()
y_val1 = X_val1['item_cnt_day']
y_val1_log = y_val1.apply(lambda x: np.log(x+2))
X_val1 = X_val1.drop(columns=['item_cnt_day'])


X_train1 = train_1[:-15000].copy()


#remove outliers
X_train1 = X_train1.sort_values(by='item_cnt_day')
X_train1 = X_train1[:-50]




y_train1 = X_train1['item_cnt_day']
y_train1_log = y_train1.apply(lambda x: np.log(x+2))
X_train1 = X_train1.drop(columns=['item_cnt_day'])

In [ ]:
y_test = pd.DataFrame(y_test)

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
cols_model = round_2_cols

my_model_2 = XGBRegressor() 
my_model_2.fit(X_train1[cols_model], y_train1_log, 
             early_stopping_rounds=5, 
             eval_set=[(X_val1[cols_model], y_val1_log)], 
             verbose=False)

prediction2 = my_model_2.predict(X_test1[cols_model])

In [ ]:
mse_2 = mean_squared_error(np.exp(prediction2)-2, y_test1)
print(mse_2**0.5)

In [ ]:
test_1 = test.merge(holidays, on=['month'])
test_1 = test.merge(red_days, on=['month', 'year'])
test_1[['shop_id', 'item_id', 'item_price', 'year', 'month', 'black_friday_x', 'new_year', 'item_price_log', 'red_day']]

In [ ]:
#SUMBMMSION
train_1 = train_1.sort_values(by=['year', 'month'])

X_train1 = train_1[-1000:].copy()

#remove outliers
X_train1 = X_train1.sort_values(by='item_cnt_day')
X_train1 = X_train1[:-100]

y_train1 = X_train1['item_cnt_day']
y_train1_log = y_train1.apply(lambda x: np.log(x+2))


X_train1 = X_train1.drop(columns=['item_cnt_day'])

In [ ]:
pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
cols_model = ['shop_id', 'item_id', 'item_price', 'year', 'month', 'black_friday_x', 'new_year', 'item_price_log', 'red_day']

round_2_cols = my_model_2 = XGBRegressor(max_depth=5,
    n_estimators=1000,
    min_child_weight=0.55, 
    colsample_bytree=0.7, 
    subsample=0.85, 
    eta=0.15) 
my_model_2.fit(X_train1[cols_model], y_train1)

test_p = test_1[cols_model]

prediction2 = my_model_2.predict(test_p)

In [ ]:
submussion = pd.DataFrame()
submussion[['ID', 'item_cnt_month']] = pd.DataFrame(prediction2).reset_index()
submussion.to_csv

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

plot_features(my_model_2, (10,14))

In [ ]:
pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv').head()

In [ ]:
categories = pd.read_csv('../input/categories-itemcsv/categories.csv')

In [ ]:
import re
pattern = r'^([\w\s]*) \-?'
get_cat = lambda x: re.match(pattern, x)[0]

categories['name'] = categories['item_category_name'].apply(get_cat)
categories